<a href="https://colab.research.google.com/github/kalvin-osoro/ml_gw/blob/main/office_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

# first neural network with keras tutorial
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

learning parameter

In [2]:
learning_param = 0.01
epochs =50

In [3]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/ml/Nairobi Office Price Ex.csv"
data = pd.read_csv(path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data.head()

,LOC,FUR,AMB,PROX_SCH,PROX_ROAD,PROX_MALL,WATER,HK_SER,SIZE,PRICE
0,karen,yes,serene,no,yes,yes,yes,yes,32.502345,31.707006
1,madaraka,yes,semi_serene,yes,yes,yes,no,no,53.426804,68.777596
2,karen,no,noisy,no,yes,yes,yes,yes,61.530358,62.562382
3,karen,yes,semi_serene,no,no,no,yes,yes,47.475640,71.546632
4,buruburu,no,semi_serene,no,yes,yes,yes,yes,59.813208,87.230925


In [ ]:
data.shape

(14, 10)

In [5]:
data.dtypes

LOC           object
FUR           object
AMB           object
PROX_SCH      object
PROX_ROAD     object
PROX_MALL     object
WATER         object
HK_SER        object
SIZE         float64
PRICE        float64
dtype: object

#data encoding

In [22]:
original_features=list(data.columns)
original_features
len(original_features)

10

In [9]:
cat_data=data.select_dtypes(include=['object']).copy()
cat_data.head()

,LOC,FUR,AMB,PROX_SCH,PROX_ROAD,PROX_MALL,WATER,HK_SER
0,karen,yes,serene,no,yes,yes,yes,yes
1,madaraka,yes,semi_serene,yes,yes,yes,no,no
2,karen,no,noisy,no,yes,yes,yes,yes
3,karen,yes,semi_serene,no,no,no,yes,yes
4,buruburu,no,semi_serene,no,yes,yes,yes,yes


In [10]:
cat_features=list(cat_data.columns)
cat_features

['LOC', 'FUR', 'AMB', 'PROX_SCH', 'PROX_ROAD', 'PROX_MALL', 'WATER', 'HK_SER']

In [25]:
nominal=['LOC','FUR','PROX_SCH','PROX_ROAD','PROX_MALL','WATER','HK_SER']
ordinal=list(set(cat_features)-set(nominal))
numerical=list(set(original_features)-set(cat_features))
target=['PRICE']
data[target]

,PRICE
0,31.707006
1,68.777596
2,62.562382
3,71.546632
4,87.230925
5,78.211518
6,79.641973
7,59.171489
8,75.331242
9,71.300880


In [26]:
for feature in ordinal:
  data[feature]=(data[feature].astype('category')).cat.codes
  

In [92]:
df_nominal=pd.get_dummies(data[nominal])
df_ordinal=data[ordinal]
df_numerical=data[numerical]
encoded_data=pd.concat([df_numerical,df_nominal,df_ordinal],axis=1)
# encoded_data=pd.concat([df_nominal,df_ordinal],axis=1)
encoded_data.head()

,SIZE,PRICE,LOC_buruburu,LOC_donholm,LOC_karen,LOC_langata,LOC_madaraka,FUR_no,FUR_yes,PROX_SCH_no,PROX_SCH_yes,PROX_ROAD_no,PROX_ROAD_yes,PROX_MALL_no,PROX_MALL_yes,WATER_no,WATER_yes,HK_SER_no,HK_SER_yes,AMB
0,32.502345,31.707006,0,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,1,2
1,53.426804,68.777596,0,0,0,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1
2,61.530358,62.562382,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0
3,47.475640,71.546632,0,0,1,0,0,0,1,1,0,1,0,1,0,0,1,0,1,1
4,59.813208,87.230925,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,1


In [93]:
encoded_data.shape

(14, 20)

end encoding

normalization

In [94]:
# encoded_data[target]
df_numerical

,SIZE,PRICE
0,32.502345,31.707006
1,53.426804,68.777596
2,61.530358,62.562382
3,47.475640,71.546632
4,59.813208,87.230925
5,55.142188,78.211518
6,52.211797,79.641973
7,39.299567,59.171489
8,48.105042,75.331242
9,52.550014,71.300880


In [95]:
num=df_numerical.to_numpy()
df_numerical

,SIZE,PRICE
0,32.502345,31.707006
1,53.426804,68.777596
2,61.530358,62.562382
3,47.475640,71.546632
4,59.813208,87.230925
5,55.142188,78.211518
6,52.211797,79.641973
7,39.299567,59.171489
8,48.105042,75.331242
9,52.550014,71.300880


In [80]:
encoded_data.shape

(14, 20)

In [91]:
# columns_titles = ["PRICE","SIZE",'']
# encoded_data=encoded_data.reindex(columns=columns_titles)

In [96]:
encoded_data.head()

,SIZE,PRICE,LOC_buruburu,LOC_donholm,LOC_karen,LOC_langata,LOC_madaraka,FUR_no,FUR_yes,PROX_SCH_no,PROX_SCH_yes,PROX_ROAD_no,PROX_ROAD_yes,PROX_MALL_no,PROX_MALL_yes,WATER_no,WATER_yes,HK_SER_no,HK_SER_yes,AMB
0,32.502345,31.707006,0,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,1,2
1,53.426804,68.777596,0,0,0,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1
2,61.530358,62.562382,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0
3,47.475640,71.546632,0,0,1,0,0,0,1,1,0,1,0,1,0,0,1,0,1,1
4,59.813208,87.230925,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,1


In [106]:
num=StandardScaler().fit_transform(df_numerical)
# y=StandardScaler().fit_transform(y)
# y
num

array([[-2.28520557, -2.72264663],
       [ 0.40190153,  0.0124287 ],
       [ 1.44255533, -0.44613097],
       [-0.36234362,  0.21672874],
       [ 1.22203988,  1.37391891],
       [ 0.62219022,  0.70846532],
       [ 0.24587098,  0.81400455],
       [-1.41231026, -0.69631171],
       [-0.28151616,  0.495958  ],
       [ 0.28930471,  0.19859708],
       [-0.62636235, -0.99186131],
       [ 0.52066778,  1.02330965],
       [-0.78761614, -0.48696528],
       [ 1.01082367,  0.50050494]])

In [111]:
encoded_data.head()

,SIZE,PRICE,LOC_buruburu,LOC_donholm,LOC_karen,LOC_langata,LOC_madaraka,FUR_no,FUR_yes,PROX_SCH_no,PROX_SCH_yes,PROX_ROAD_no,PROX_ROAD_yes,PROX_MALL_no,PROX_MALL_yes,WATER_no,WATER_yes,HK_SER_no,HK_SER_yes,AMB
0,32.502345,31.707006,0,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,1,2
1,53.426804,68.777596,0,0,0,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1
2,61.530358,62.562382,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0
3,47.475640,71.546632,0,0,1,0,0,0,1,1,0,1,0,1,0,0,1,0,1,1
4,59.813208,87.230925,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,1


,SIZE,PRICE,LOC_buruburu,LOC_donholm,LOC_karen,LOC_langata,LOC_madaraka,FUR_no,FUR_yes,PROX_SCH_no,PROX_SCH_yes,PROX_ROAD_no,PROX_ROAD_yes,PROX_MALL_no,PROX_MALL_yes,WATER_no,WATER_yes,HK_SER_no,HK_SER_yes,AMB
0,32.502345,31.707006,0,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,1,2
1,53.426804,68.777596,0,0,0,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1
2,61.530358,62.562382,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0
3,47.475640,71.546632,0,0,1,0,0,0,1,1,0,1,0,1,0,0,1,0,1,1
4,59.813208,87.230925,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,1


Normalization


In [112]:
cols_to_norm = ['SIZE','PRICE']
encoded_data[cols_to_norm] = encoded_data[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [126]:
df= encoded_data

In [127]:
df

,SIZE,PRICE,LOC_buruburu,LOC_donholm,LOC_karen,LOC_langata,LOC_madaraka,FUR_no,FUR_yes,PROX_SCH_no,PROX_SCH_yes,PROX_ROAD_no,PROX_ROAD_yes,PROX_MALL_no,PROX_MALL_yes,WATER_no,WATER_yes,HK_SER_no,HK_SER_yes,AMB
0,0.000000,0.000000,0,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,1,2
1,0.720837,0.667651,0,0,0,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1
2,1.000000,0.555713,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0
3,0.515822,0.717522,0,0,1,0,0,0,1,1,0,1,0,1,0,0,1,0,1,1
4,0.940845,1.000000,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,1
5,0.779931,0.837558,0,1,0,0,0,1,0,1,0,1,0,0,1,1,0,0,1,2
6,0.678980,0.863321,0,0,0,1,0,1,0,0,1,0,1,1,0,1,0,0,1,3
7,0.234161,0.494642,0,0,0,1,0,0,1,1,0,1,0,0,1,0,1,1,0,2
8,0.537505,0.785684,0,1,0,0,0,0,1,0,1,1,0,1,0,0,1,1,0,1
9,0.690632,0.713096,0,0,1,0,0,0,1,1,0,1,0,1,0,1,0,1,0,2


In [129]:
# def swap_columns(encoded_data, SIZE, PRICE):
#     col_list = list(encoded_data.columns)
#     X, y = col_list.index(SIZE), col_list.index(PRICE)
#     col_list[y], col_list[X] = col_list[X], col_list[y]
#     df = encoded_data[col_list]
#     return df
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

df = swap_columns(df, 'points', 'rebounds'):

In [149]:
# Changing pandas dataframe to numpy array
X = df.iloc[:,1:20].values
# y = df.iloc[:,:1].values
y=df['PRICE']

In [150]:
y

0     0.000000
1     0.667651
2     0.555713
3     0.717522
4     1.000000
5     0.837558
6     0.863321
7     0.494642
8     0.785684
9     0.713096
10    0.422497
11    0.914414
12    0.545745
13    0.786794
Name: PRICE, dtype: float64

In [136]:
X

array([[0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 1.        , 1.        , 0.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        1.        , 0.        , 1.        , 2.        ],
       [0.66765082, 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 1.        , 0.        , 1.        ,
        0.        , 1.        , 0.        , 1.        , 1.        ,
        0.        , 1.        , 0.        , 1.        ],
       [0.55571323, 0.        , 0.        , 1.        , 0.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 1.        , 0.        , 1.        , 0.        ,
        1.        , 0.        , 1.        , 0.        ],
       [0.71752187, 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 1.        , 1.        , 0.        ,
        1.        , 0.        , 1.        , 0.        , 0.        ,
        1.   

In [152]:
df.shape

(14, 20)

In [153]:
...
# define the keras model
model = Sequential()
model.add(Dense(12, input_shape=(19,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
...

Ellipsis

In [154]:
# compile the keras model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [155]:
# fit the keras model on the dataset
model.fit(X, y, epochs=5, batch_size=10)

Epoch 1/5
2/2 [==============================] - 1s 6ms/step - loss: 0.1251 - accuracy: 0.0714
Epoch 2/5
2/2 [==============================] - 0s 4ms/step - loss: 0.1208 - accuracy: 0.0714
Epoch 3/5
2/2 [==============================] - 0s 6ms/step - loss: 0.1174 - accuracy: 0.0714
Epoch 4/5
2/2 [==============================] - 0s 5ms/step - loss: 0.1141 - accuracy: 0.0714
Epoch 5/5
2/2 [==============================] - 0s 5ms/step - loss: 0.1114 - accuracy: 0.0714


In [156]:

# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

1/1 [==============================] - 0s 136ms/step - loss: 0.1088 - accuracy: 0.0714
Accuracy: 7.14


#archive

In [ ]:
(train_data, train_targets) = data
#  (test_data, test_targets) =  data


In [ ]:
# X = data[:,1:9]

# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean',verbose=0)
# imputer = imputer.fit(data.iloc[:, 1:9])
# X.iloc[:, 1:9] = imputer.transform(X.iloc[:, 1:9])

X = data.iloc[:,1:9].values
y = data.iloc[:,9]

In [ ]:
X

array([['yes', 'serene', 'no', 'yes', 'yes', 'yes', 'yes', 32.50234527],
       ['yes', 'semi_serene', 'yes', 'yes', 'yes', 'no', 'no',
        53.42680403],
       ['no', 'noisy', 'no', 'yes', 'yes', 'yes', 'yes', 61.53035803],
       ['yes', 'semi_serene', 'no', 'no', 'no', 'yes', 'yes',
        47.47563963],
       ['no', 'semi_serene', 'no', 'yes', 'yes', 'yes', 'yes',
        59.81320787],
       ['no', 'serene', 'no', 'no', 'yes', 'no', 'yes', 55.14218841],
       ['no', 'very_noisy', 'yes', 'yes', 'no', 'no', 'yes', 52.21179669],
       ['yes', 'serene', 'no', 'no', 'yes', 'yes', 'no', 39.29956669],
       ['yes', 'semi_serene', 'yes', 'no', 'no', 'yes', 'no',
        48.10504169],
       ['yes', 'serene', 'no', 'no', 'no', 'no', 'no', 52.55001444],
       ['yes', 'noisy', 'yes', 'yes', 'no', 'yes', 'yes', 45.41973014],
       ['no', 'semi_serene', 'yes', 'no', 'yes', 'yes', 'yes',
        54.35163488],
       ['yes', 'semi_serene', 'yes', 'yes', 'no', 'no', 'no', 44.1640495],
 

In [ ]:

X = np.asarray(X).astype(np.float32)

ValueError: ignored

In [ ]:
y

0     31.707006
1     68.777596
2     62.562382
3     71.546632
4     87.230925
5     78.211518
6     79.641973
7     59.171489
8     75.331242
9     71.300880
10    55.165677
11    82.478847
12    62.008923
13    75.392870
Name: PRICE, dtype: float64

In [ ]:
...
# define the keras model
model = Sequential()
model.add(Dense(12, input_shape=(9,), activation='relu',  name="layer1"))
model.add(Dense(8, activation='relu',  name="layer2"))
model.add(Dense(1,   name="layer3"))
...



Ellipsis

In [ ]:
# # Define Sequential model with 3 layers
# model = keras.Sequential(
#     [
#         layers.Dense(12, input_shape=(9,), activation="relu", name="layer1"),
#         layers.Dense(8, activation="relu", name="layer2"),
#         layers.Dense(1, name="layer3"),
#     ]
# )


In [ ]:

...
# compile the keras model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
...

In [ ]:
# fit the keras model on the dataset
model.fit(X, y, epochs=5, batch_size=10)

ValueError: ignored